# Fase 2: Limpieza y Transformación

#### 1. Importamos las librerías necesarias y agregamos la ruta de nuestro archivo .py 

In [297]:
import sys
import pandas as pd

# Agregamos la ruta
sys.path.append('../')

# Importamos la función procesar_dataframe de limpieza.py
from src.limpieza import procesar_dataframe

# Importamos los dos diccionarios de variables.py
from src.variables import cambiar_nombre_columnas, cambiar_tipo_columnas

#### 2. Cargamos el dataframe transformado, hacemos la llamada a la función y comprobamos los cambios

In [298]:
# Cargamos el DataFrame
df_csv = pd.read_csv('../data/transform_data/transform_Data.csv', low_memory=False)

# Llamamos a la función procesar_dataframe con los diccionarios importados
df = procesar_dataframe(df_csv, cambiar_nombre_columnas, cambiar_tipo_columnas)

# Verificamos el resultado mostrando las primeras filas
df.dtypes

Unnamed: 0_x                   float64
age                              int64
job                             object
marital                         object
education                       object
default                           bool
housing                           bool
loan                              bool
contact                         object
duration                       float64
campaign                       float64
pdays                          float64
previous                       float64
poutcome                        object
emp_var_rate                   float64
cons_price_idx                 float64
cons_conf_idx                  float64
euribor3m                      float64
nr_employed                    float64
y                               object
date                    datetime64[ns]
latitude                       float64
longitude                      float64
id_                     string[python]
Unnamed: 0_y                     int64
Income                   

#### 3. Analizamos si existen o no valores duplicados, para poder reducir el tamaño del archivo y mejorar la velocidad del fichero

In [299]:
# Comprobamos si existen valores duplicados
if df.duplicated().values.any():
    print("Existen valores duplicados por eliminar")
else:
    print("No existen valores duplicados")

# Comprobamos y verificamos que no existen valores duplicados
df.duplicated().sum()

No existen valores duplicados


np.int64(0)

#### 4. Analizamos si existen o no valores nulos

In [300]:
# Calculamos cuantos valores nulos existen
print("Los valores nulos existentes por columna son:")
df.isnull().sum()

Los valores nulos existentes por columna son:


Unnamed: 0_x              170
age                         0
job                       515
marital                   255
education                1977
default                     0
housing                     0
loan                        0
contact                   170
duration                  170
campaign                  170
pdays                     170
previous                  170
poutcome                  170
emp_var_rate              170
cons_price_idx            641
cons_conf_idx             170
euribor3m                9426
nr_employed               170
y                         170
date                    43170
latitude                  170
longitude                 170
id_                         0
Unnamed: 0_y                0
Income                      0
Kidhome                     0
Teenhome                    0
Dt_Customer                 0
Num_Web_Visits_Month        0
dtype: int64

In [301]:
# Calculamos el % de nulos en cada columna, para saber la cantidad de nulos que hemos obtenido
df.isnull().mean() * 100

Unnamed: 0_x              0.393792
age                       0.000000
job                       1.192958
marital                   0.590688
education                 4.579569
default                   0.000000
housing                   0.000000
loan                      0.000000
contact                   0.393792
duration                  0.393792
campaign                  0.393792
pdays                     0.393792
previous                  0.393792
poutcome                  0.393792
emp_var_rate              0.393792
cons_price_idx            1.484827
cons_conf_idx             0.393792
euribor3m                21.834607
nr_employed               0.393792
y                         0.393792
date                    100.000000
latitude                  0.393792
longitude                 0.393792
id_                       0.000000
Unnamed: 0_y              0.000000
Income                    0.000000
Kidhome                   0.000000
Teenhome                  0.000000
Dt_Customer         

#### 5. Conclusiones del cálculo de % de valores nulos

**Calculamos el % de valores nulos en cada columna, y obtenemos las siguientes conclusiones:**

1. Si una columna contiene el 100% de valores nulos, lo más eficiente es eliminar dicha columna (date)<br>

2. Si menos del 5% de los valores de la columna son nulos, podemos sustituirlos por la mediana en columnas numéricas y la moda en columnas categóricas.<br>

3. Sustituir por la moda es adecuado para las columnas categóricas, ya que la moda es el valor más frecuente.<br>

4. Sustituir por la mediana es útil cuando la distribución de los datos es asimétrica (sesgada), ya que la mediana no se ve afectada por los valores extremos.<br>

In [302]:
# Según el análisis eliminamos la columna Date, puesto que tiene el 100% de valores nulos
df = df.drop('date', axis=1)

In [303]:
# Sustituimos los nulos en las columnas numéricas, por la mediana de cada columna
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

In [304]:
# Sustituimos los nulos en las columnas categóricas, por la moda de cada columna
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])

In [305]:
# Verificamos que ya no hay valores nulos
df.isnull().sum()

Unnamed: 0_x            0
age                     0
job                     0
marital                 0
education               0
default                 0
housing                 0
loan                    0
contact                 0
duration                0
campaign                0
pdays                   0
previous                0
poutcome                0
emp_var_rate            0
cons_price_idx          0
cons_conf_idx           0
euribor3m               0
nr_employed             0
y                       0
latitude                0
longitude               0
id_                     0
Unnamed: 0_y            0
Income                  0
Kidhome                 0
Teenhome                0
Dt_Customer             0
Num_Web_Visits_Month    0
dtype: int64

In [306]:
# Verificamos que se aplicaron los cambios
df.head()

,Unnamed: 0_x,age,job,marital,education,default,housing,loan,contact,duration,...,y,latitude,longitude,id_,Unnamed: 0_y,Income,Kidhome,Teenhome,Dt_Customer,Num_Web_Visits_Month
0,36832.0,22,services,SINGLE,high.school,False,True,False,telephone,94.0,...,no,37.753,-110.119,0000e811-006e-4404-b535-89bf6cd96553,12122,101916,2,0,2014-07-17,3
1,40976.0,56,technician,DIVORCED,professional.course,False,False,False,cellular,464.0,...,yes,27.766,-89.350,0000ea53-e9b2-4b3f-9f4b-058f37e5fab8,11896,57990,2,2,2014-01-04,3
2,33283.0,31,blue-collar,MARRIED,basic.9y,False,False,False,cellular,365.0,...,no,36.347,-69.175,000165f9-20c0-4cb5-bd47-6233b92655c1,4203,175137,1,1,2014-12-01,8
3,2696.0,38,blue-collar,SINGLE,university.degree,False,False,False,telephone,109.0,...,no,26.893,-68.620,00024507-c59b-4eee-86d5-cc341b96eb6d,2696,62489,2,0,2012-01-25,7
4,37890.0,39,entrepreneur,MARRIED,basic.6y,False,True,False,cellular,265.0,...,yes,48.901,-96.742,0004e1d1-958d-4abf-a57c-9b9c7be887a0,8810,169187,2,2,2014-05-12,16


#### 6. Analizamos las columnas categóricas

In [307]:
# Verificamos posibles erratas o valores mal escritos, con head mostramos los 20 valores de mayor frecuencia
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n🔹 Columna: {col}")
    print(df_csv[col].value_counts(dropna=False).head(20))


🔹 Columna: job
job
admin.           10873
blue-collar       9654
technician        7026
services          4162
management        3050
retired           1790
entrepreneur      1522
self-employed     1489
housemaid         1123
unemployed        1063
student            903
NaN                515
Name: count, dtype: int64

🔹 Columna: marital
marital
MARRIED     25999
SINGLE      12105
DIVORCED     4811
NaN           255
Name: count, dtype: int64

🔹 Columna: education
education
university.degree      12722
high.school             9925
basic.9y                6309
professional.course     5477
basic.4y                4356
basic.6y                2386
NaN                     1977
illiterate                18
Name: count, dtype: int64

🔹 Columna: contact
contact
cellular     27396
telephone    15604
NaN            170
Name: count, dtype: int64

🔹 Columna: poutcome
poutcome
NONEXISTENT    37103
FAILURE         4461
SUCCESS         1436
NaN              170
Name: count, dtype: int64

🔹 Columna:

In [308]:
# Eliminamos espacios en blanco, espacios dobles o caracteres
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace(r'\s+', ' ', regex=True)

#### 7. Analizamos las columnas numéricas

In [309]:
# Revisamos las estadísticas generales
df.select_dtypes(include=["number"]).describe()

,Unnamed: 0_x,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,latitude,longitude,Unnamed: 0_y,Income,Kidhome,Teenhome,Num_Web_Visits_Month
count,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000,43170.000000
mean,20651.065184,35.078365,257.429210,2.564999,962.475353,0.173338,0.081156,93.576814,-40.514132,3.887375,5166.947693,36.856320,-95.938911,7916.022469,93227.389622,1.004749,0.998633,16.591290
std,11845.378197,16.353714,258.203313,2.767058,186.905423,0.496505,1.572102,0.575618,4.628752,1.619048,72.284229,7.211708,16.719264,5228.248885,50498.181989,0.815996,0.816008,9.241769
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,24.396000,-124.997000,0.000000,5841.000000,0.000000,0.000000,1.000000
25%,10420.250000,30.000000,103.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.479000,5099.100000,30.633250,-110.440000,3597.000000,49608.000000,0.000000,0.000000,9.000000
50%,20642.500000,36.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000,36.761000,-95.899500,7194.500000,93009.500000,1.000000,1.000000,17.000000
75%,30889.750000,46.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.959000,5228.100000,43.089000,-81.478500,11705.750000,136740.500000,2.000000,2.000000,25.000000
max,41187.000000,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,49.384000,-66.937000,20114.000000,180802.000000,2.000000,2.000000,32.000000


In [310]:
# Detectamos valores fuera de rango, por ejemplo si obtenemos una edad negativa
for col in df.select_dtypes(include=['number']).columns:
    print(f"\n🔹 Columna: {col}")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Máximo: {df[col].max()}, Mínimo: {df[col].min()}")


🔹 Columna: Unnamed: 0_x
Valores únicos: 41033
Máximo: 41187.0, Mínimo: 0.0

🔹 Columna: age
Valores únicos: 79
Máximo: 98, Mínimo: 0

🔹 Columna: duration
Valores únicos: 1540
Máximo: 4918.0, Mínimo: 0.0

🔹 Columna: campaign
Valores únicos: 42
Máximo: 56.0, Mínimo: 1.0

🔹 Columna: pdays
Valores únicos: 27
Máximo: 999.0, Mínimo: 0.0

🔹 Columna: previous
Valores únicos: 8
Máximo: 7.0, Mínimo: 0.0

🔹 Columna: emp_var_rate
Valores únicos: 10
Máximo: 1.4, Mínimo: -3.4

🔹 Columna: cons_price_idx
Valores únicos: 26
Máximo: 94.767, Mínimo: 92.201

🔹 Columna: cons_conf_idx
Valores únicos: 26
Máximo: -26.9, Mínimo: -50.8

🔹 Columna: euribor3m
Valores únicos: 309
Máximo: 5.045, Mínimo: 0.634

🔹 Columna: nr_employed
Valores únicos: 11
Máximo: 5228.1, Mínimo: 4963.6

🔹 Columna: latitude
Valores únicos: 20492
Máximo: 49.384, Mínimo: 24.396

🔹 Columna: longitude
Valores únicos: 30449
Máximo: -66.937, Mínimo: -124.997

🔹 Columna: Unnamed: 0_y
Valores únicos: 20115
Máximo: 20114, Mínimo: 0

🔹 Columna: I

#### 8. Cálculo del Método de Rango Intercuartílico (IQR)

In [311]:
# Detectamos valores bajos o altos que podrían causar conflicto
import numpy as np

for col in df.select_dtypes(include=['number']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    print(f"\n🔹 Columna: {col} - Outliers encontrados: {outliers.shape[0]}")


🔹 Columna: Unnamed: 0_x - Outliers encontrados: 0

🔹 Columna: age - Outliers encontrados: 5691

🔹 Columna: duration - Outliers encontrados: 3121

🔹 Columna: campaign - Outliers encontrados: 2504

🔹 Columna: pdays - Outliers encontrados: 1588

🔹 Columna: previous - Outliers encontrados: 5897

🔹 Columna: emp_var_rate - Outliers encontrados: 0

🔹 Columna: cons_price_idx - Outliers encontrados: 0

🔹 Columna: cons_conf_idx - Outliers encontrados: 477

🔹 Columna: euribor3m - Outliers encontrados: 0

🔹 Columna: nr_employed - Outliers encontrados: 0

🔹 Columna: latitude - Outliers encontrados: 0

🔹 Columna: longitude - Outliers encontrados: 0

🔹 Columna: Unnamed: 0_y - Outliers encontrados: 0

🔹 Columna: Income - Outliers encontrados: 0

🔹 Columna: Kidhome - Outliers encontrados: 0

🔹 Columna: Teenhome - Outliers encontrados: 0

🔹 Columna: Num_Web_Visits_Month - Outliers encontrados: 0


#### 9. Guardamos el CSV con la limpieza final realizada

In [312]:
df.to_csv('../data/transform_data/bank_clean.csv', index=False)